##1. Descarga y exploración del corpus
El alumno descargará el/los corpus que desee (detalle sobre los datos a utilizar en
el siguiente punto) y realizará un análisis exploratorio de los datos.
Este ejercicio deberá contener:

- Cardinalidad del vocabulario
- Distribución de reviews por número de estrellas
- No de reviews positivas y negativas
- N-grams más frecuentes
- Nubes de palabras
- Visualización en 2 dimensiones de algunos word embeddings calculados con
Word2Vec (elegir 4-5 palabras y pintar las top 10 más similares)
- Conclusiones de la exploración
- Cualquier otra métrica / exploración / cálculo que el alumno considere

In [1]:
!pip install -r requirements.txt
!pip install utils.py

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user


In [ ]:
import random
import numpy as np
import pandas as pd
from collections import Counter # Exploración
from nltk import ngrams # Exploración
from nltk.probability import FreqDist # Exploración


In [ ]:
#importamos librerias necesarias
import pandas as pd

#Cargo Dataset

df= pd.read_csv('DatafinitiElectronicsProductData.csv')

In [ ]:
#hagp un pequeña prebisualizaicion de los datos
df.head()


In [ ]:
df.shape

In [ ]:
#eliminamos las columnas que tengas algun na
df = df.dropna(subset= ['reviews.text','reviews.rating'])
df.shape

In [ ]:
#Convierto el rating a entero para mas seguridad
df['reviews.rating']= df['reviews.rating'].astype(int)

In [ ]:
aux_df = pd.DataFrame(df['reviews.rating'].value_counts(sort=False).sort_index())
aux_df = aux_df.plot(kind='bar', title='Overall sentiment distribution (review)', figsize=(8, 4))

In [ ]:
print(df.head())
print(df['reviews.rating'].value_counts().sort_values())

In [ ]:
#Voy a definir una funcion que clasifique el sentimiento para una clasificacion binaria, siendo el cero negativo y 1 positivo

def rating_to_sentiment(rating):
  """
  Tranforma una clasificacion de de 5 niveles en 2 positivo o negativo
  """
  if rating >=4:
    return 1
  else:
    return 0

In [ ]:
import spacy

# Cargar modelo de spaCy en inglés
nlp = spacy.load('en_core_web_sm')

#Creo una funcino de limpieza. Para las stopwords usare Spacy
def clean_text_spacy(text):
    """
    Preprocesa texto con spaCy:
    - Minúsculas
    - Tokenización, lematización
    - Quitar signos, stopwords y palabras muy cortas
    """
    doc = nlp(str(text).lower())
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and
           not token.is_punct and
           not token.like_url and
           not token.is_space and
           token.is_alpha and
           len(token) > 2
    ]
    return ' '.join(tokens)

In [ ]:
#Creo una nueva columna de sentimiento
df['sentiment'] = df['reviews.rating'].apply(rating_to_sentiment)

In [ ]:
#Comprobamos
print(df.head())
print(df['sentiment'].value_counts())

In [ ]:
aux_df2 = df['sentiment'].value_counts(sort=False).sort_index()
aux_df2 = aux_df2.plot(kind='bar', title='Sentiment distribution (Corpus Final)', figsize=(8, 4))

In [ ]:
# Aplicar la función a la columna de reseñas
df['clean_review'] = df['reviews.text'].apply(clean_text_spacy)

# Mostrar ejemplo de resultados
print(df[['reviews.text', 'clean_review']].head(3))

In [ ]:
df.head()

In [ ]:
#Funcion para ver los n gramas mas frecuentes
def get_top_ngrams(corpus, ngram_range=(1,1), top_n=10):
    """
    Recibe un texto, los ngramas y el numero de ngramas que queremos ver
    Devuelve los N-grams más frecuentes en el corpus de texto.

    """
    vec = CountVectorizer(ngram_range=ngram_range)
    X = vec.fit_transform(corpus)
    sum_words = X.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:top_n]

In [ ]:
# Top 10 palabras
top_uni = get_top_ngrams(df['clean_review'], ngram_range=(1,1), top_n=10)
print("Top 10 unigramas:", top_uni)

# Top 10 bigramas
top_bi = get_top_ngrams(df['clean_review'], ngram_range=(2,2), top_n=10)
print("Top 10 bigramas:", top_bi)

# Top 10 trigramas
top_tri = get_top_ngrams(df['clean_review'], ngram_range=(3,3), top_n=10)
print("Top 10 trigramas:", top_tri)

In [ ]:
import matplotlib.pyplot as plt
#Funcion para pintar los n gramas mas frecuentes
def plot_ngrams(ngrams, title):
    words = [w for w, c in ngrams]
    counts = [c for w, c in ngrams]
    plt.figure(figsize=(8,4))
    plt.barh(words[::-1], counts[::-1], color='steelblue')
    plt.title(title)
    plt.xlabel('Frecuencia')
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizar bigramas
plot_ngrams(top_uni, 'Top 10 gramas más frecuentes')
plot_ngrams(top_bi, 'Top 10 Bigramas más frecuentes')
plot_ngrams(top_tri, 'Top 10 Trigramas más frecuentes')

In [ ]:
#Voy visulaizar lo mismo pero segun el sentimiento

# Top 10 palabras
top_uni_pos = get_top_ngrams(df[df['sentiment']==1]['clean_review'], ngram_range=(1,1), top_n=10)
print("Top 10 unigramas:", top_uni_pos)

# Top 10 bigramas
top_bi_pos = get_top_ngrams(df[df['sentiment']==1]['clean_review'], ngram_range=(2,2), top_n=10)
print("Top 10 bigramas:", top_bi_pos)

# Top 10 trigramas
top_tri_pos = get_top_ngrams(df[df['sentiment']==1]['clean_review'], ngram_range=(3,3), top_n=10)
print("Top 10 trigramas:", top_tri_pos)

plot_ngrams(top_uni_pos, 'Top 10 gramas más frecuentes pos')
plot_ngrams(top_bi_pos, 'Top 10 Bigramas más frecuentes pos')
plot_ngrams(top_tri, 'Top 10 Trigramas más frecuentes pos')

In [ ]:
# Top 10 palabras
top_uni_neg = get_top_ngrams(df[df['sentiment']==0]['clean_review'], ngram_range=(1,1), top_n=10)
print("Top 10 unigramas:", top_uni)

# Top 10 bigramas
top_bi_neg = get_top_ngrams(df[df['sentiment']==0]['clean_review'], ngram_range=(2,2), top_n=10)
print("Top 10 bigramas:", top_bi)

# Top 10 trigramas
top_tri_neg = get_top_ngrams(df[df['sentiment']==0]['clean_review'], ngram_range=(3,3), top_n=10)
print("Top 10 trigramas:", top_tri)

plot_ngrams(top_uni_neg, 'Top 10 gramas más frecuentes neg')
plot_ngrams(top_bi_neg, 'Top 10 Bigramas más frecuentes neg')
plot_ngrams(top_tri, 'Top 10 Trigramas más frecuentes neg')

In [ ]:
pip install wordcloud

In [ ]:
#Biblioteca para nube de palabras y representacion
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
#nube de palabras en general
text = ' '.join(df['clean_review'])

wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras - Reseñas Amazon Electronics')
plt.show()

In [ ]:
#nube de palabras con sentimiento positivo
text_pos = ' '.join(df[df['sentiment']==1]['clean_review'])

wordcloud_pos = WordCloud(width=800, height=400, background_color='white', colormap='Blues').generate(text_pos)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras - Reseñas Positivas')
plt.show()

In [ ]:
#nube de palabras con sentimiento positivo
text_pos = ' '.join(df[df['sentiment']==1]['clean_review'])

wordcloud_pos = WordCloud(width=800, height=400, background_color='white', colormap='Blues').generate(text_pos)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras - Reseñas Positivas')
plt.show()

In [ ]:
# Top 30 palabras por clase (puedes cambiar el número según tu corpus)
top_n = 30
uni_pos = get_top_ngrams(df[df['sentiment']==1]['clean_review'], ngram_range=(1,1), top_n=top_n)
uni_neg = get_top_ngrams(df[df['sentiment']==0]['clean_review'], ngram_range=(1,1), top_n=top_n)

# Extraer solo las palabras (sin frecuencia)
words_pos = set([w for w, _ in uni_pos])
words_neg = set([w for w, _ in uni_neg])

# Palabras comunes
common_words = words_pos.intersection(words_neg)
print("Palabras comunes más frecuentes:", common_words)


In [ ]:
pip install gensim


In [ ]:
from gensim.models import Word2Vec

# Tokenizar el corpus limpio en listas de palabras
# Cada reseña será una lista de tokens
tokenized_reviews = [review.split() for review in df['clean_review']]

# Entrenar el modelo Word2Vec (dimensión 100, puedes ajustar)
w2v_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=3, workers=4, seed=42)


In [ ]:
tokenized_reviews[0]

In [ ]:
df['clean_review'][0]

In [ ]:
palabras_clave = ["good", "device", "price", "battery", "computer"]

for palabra in palabras_clave:
    if palabra in w2v_model.wv:
        similares = w2v_model.wv.most_similar(palabra, topn=10)
        print(f"\nPalabras más similares a '{palabra}':")
        for similar, score in similares:
            print(f"  {similar:12} ({score:.2f})")
    else:
        print(f"\nLa palabra '{palabra}' no está en el vocabulario del modelo.")

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

def plot_word2vec_neighbors(w2v_model, palabras_clave, n_vecinos=10):
    palabras_plot = []
    vectores = []
    colores = []

    for palabra in palabras_clave:
        if palabra in w2v_model.wv:
            palabras_plot.append(palabra)
            vectores.append(w2v_model.wv[palabra])
            colores.append('red')
            # Añade los vecinos
            vecinos = w2v_model.wv.most_similar(palabra, topn=n_vecinos)
            for vecino, _ in vecinos:
                if vecino not in palabras_plot:
                    palabras_plot.append(vecino)
                    vectores.append(w2v_model.wv[vecino])
                    colores.append('blue')

    # Reducción de dimensionalidad a 2D
    tsne = TSNE(n_components=2, random_state=42, perplexity=15)
    emb_2d = tsne.fit_transform(np.array(vectores))

    plt.figure(figsize=(12,8))
    for i, palabra in enumerate(palabras_plot):
        plt.scatter(emb_2d[i,0], emb_2d[i,1], color=colores[i])
        plt.text(emb_2d[i,0]+0.3, emb_2d[i,1], palabra, fontsize=12)
    plt.title("Visualización 2D de embeddings Word2Vec\nPalabras clave y sus vecinos")
    plt.axis('off')
    plt.show()

plot_word2vec_neighbors(w2v_model, palabras_clave, n_vecinos=10)

5. Conclusiones del Análisis Exploratorio
El análisis exploratorio del corpus de reseñas de productos electrónicos de Amazon ha revelado varios aspectos fundamentales para el modelado de sentimiento:

Desbalance de clases: La gran mayoría de las reseñas son positivas, lo que refleja la tendencia habitual en portales de comercio electrónico. Este desbalance impone la necesidad de emplear métricas apropiadas (precision, recall, F1-score por clase) y posiblemente ajustar los pesos en los modelos supervisados.

Patrones léxicos: El análisis de N-grams muestra que existen frases y palabras recurrentes que capturan tanto elogios (“easy use”, “battery life”, “sound quality”) como quejas (“stop work”, “waste money”, “not work”). Muchas palabras altamente frecuentes son comunes a ambos sentimientos y pueden ser consideradas como stopwords de dominio para mejorar la discriminación en modelos.

Visualización con wordclouds: Las nubes de palabras refuerzan que ciertos términos técnicos y adjetivos como “great”, “easy”, “battery”, “quality”, “work” predominan en el lenguaje de los usuarios. Al excluir palabras comunes, surgen diferencias léxicas más informativas entre reseñas positivas y negativas.

Relaciones semánticas con Word2Vec: El embedding Word2Vec entrenado sobre el corpus permite identificar asociaciones semánticas y sinónimos no evidentes solo a partir de frecuencia. Palabras como “good”, “price”, “battery” tienen vecinos semánticos coherentes con el dominio, lo que puede enriquecer futuros modelos si se emplean embeddings.

Otras métricas: La longitud de las reseñas y la diversidad de vocabulario también presentan diferencias leves por clase, pero no son tan determinantes como los patrones de N-grams o las relaciones captadas por embeddings.

En síntesis, el corpus contiene señales claras y diferenciables en el plano léxico y semántico, pese a la predominancia de sentimiento positivo. El análisis exploratorio permite seleccionar features relevantes, construir stopwords personalizadas y justificar el diseño de modelos supervisados robustos para la siguiente etapa.